# Observer Pattern

In [1]:
import inspect
from typing import Callable, Type

def override(interface_class: Type) -> Callable:
    def decorator(method: Callable) -> Callable:
        method_name = method.__name__
        for cls in inspect.getmro(interface_class):
            if cls is not object and method_name in cls.__dict__:
                return method
        raise TypeError(f"{method_name} does not override any method in {interface_class.__name__}")
    return decorator

In [15]:
from abc import ABC, abstractmethod
from typing import List

class Observer(ABC):

    def __init__(self) -> None:
        self._observables: List['Observable'] = []

    def getObservables(self) -> List['Observable']:
        for observable in self._observables:
            print(observable)

    def observe(self, observable: 'Observable') -> None:
        observable.registerObserver(self)
        self._observables.append(observable)

    def unobserve(self, observable: 'Observable') -> None:
        observable.unregisterObserver(self)
        self._observables.remove(observable)

    @abstractmethod
    def update(self, observable: 'Observable'):
        raise NotImplementedError

class Observable(ABC):

    def __init__(self) -> None:
        self._observers: List[Observer] = []

    def getObservers(self) -> List[Observer]:
        for observer in self._observers:
            print(observer)

    def registerObserver(self, observer: Observer) -> None:
        self._observers.append(observer)
    
    def unregisterObserver(self, observer: Observer) -> None:
        self._observers.remove(observer)
    
    @abstractmethod
    def notifyObservers(self) -> None:
        raise NotImplementedError

In [20]:
class WeatherData(Observable):

    def __init__(self, temperature: float, humidity: float, pressure: float) -> None:
        super().__init__()
        self._temperature = temperature
        self._humidity = humidity
        self._pressure = pressure

    def __repr__(self) -> str:
        return f"{self.__class__.__name__}(Temperature: {self.getTemperatue()}, Humidity: {self.getHumidity()}, Pressure: {self.getPressure()})"

    def getTemperatue(self) -> float:
        return self._temperature
    
    def getHumidity(self) -> float:
        return self._humidity
    
    def getPressure(self) -> float:
        return self._pressure
    
    def setTemperatue(self, temperature: float) -> None:
        self._temperature = temperature

    def setHumidity(self, humidity: float) -> None:
        self._humidity = humidity

    def setPressure(self, pressure: float) -> None:
        self._pressure = pressure

    @override(Observable)
    def notifyObservers(self) -> None:
        for observer in self._observers:
            observer.update(self)

    def measurementChanged(self) -> None:
        self.notifyObservers()

    def setMeasurements(self, temperature: float, humidity: float, pressure: float) -> None:
        self.setTemperatue(temperature)
        self.setHumidity(humidity)
        self.setPressure(pressure)
        self.measurementChanged()

class SensorData(Observable):

    def __init__(self, data: float) -> None:
        super().__init__()
        self._data = data

    def __repr__(self) -> str:
        return f"{self.__class__.__name__}(Data: {self.getData()})"

    def getData(self) -> float:
        return self._data
    
    def setData(self, data: float) -> None:
        self._data = data

    @override(Observable)
    def notifyObservers(self) -> None:
        for observer in self._observers:
            observer.update(self)

    def dataChanged(self) -> None:
        self.notifyObservers()

    def setMeasurements(self, data: float) -> None:
        self.setData(data)
        self.dataChanged()

In [21]:
class WeatherDisplay(Observer):

    def __init__(self) -> None:
        super().__init__()

    @override(Observer)
    def update(self, observable: Observable):
        print(observable)

In [24]:
weatherData = WeatherData(0.1, 0.5, 0.5)
sensorData = SensorData(0.1)

weatherDisplay = WeatherDisplay()
weatherDisplay.observe(weatherData)
weatherDisplay.observe(sensorData)

weatherData.setMeasurements(0.2, 0.2, 0.2)
sensorData.setMeasurements(0.2)

WeatherData(Temperature: 0.2, Humidity: 0.2, Pressure: 0.2)
SensorData(Data: 0.2)
